# [Metaprogramming](https://docs.julialang.org/en/latest/manual/metaprogramming/#)

A "meta" program is a program that manipulates programs.

Most common use refers to a program that generates another program.

Julia allows us to talk in a "meta" way ("one level up"), about Julia code, that is to **"treat code as data"** and manipulate it as just another object in Julia.

## Symbols and Expressions

In [146]:
:a

:a

In [147]:
typeof(:a)

Symbol

In [148]:
eval(:a)

UndefVarError: UndefVarError: a not defined

In [149]:
a = 2

2

In [150]:
eval(:a)

2

In [151]:
typeof(:+)

Symbol

In [152]:
typeof(:sin)

Symbol

Symbols may be combined into *expressions*, which are the basic objects that represent pieces of Julia code:

In [153]:
ex = :(a + b)  # the expression 'a + b'

:(a + b)

In [154]:
typeof(ex)

Expr

In [155]:
b = 7
eval(ex)

9

An expression is just a Julia object, so we can introspect (find out information about it):

In [156]:
dump(ex)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Symbol a
    3: Symbol b


In [157]:
dump(:(x = 3))

Expr
  head: Symbol =
  args: Array{Any}((2,))
    1: Symbol x
    2: Int64 3


The job of Julia's parser is to convert a sequence of characters into these `Expr` objects:

In [159]:
Meta.parse("a + b")

:(a + b)

More complicated expressions are represented as **"abstract syntax trees" (ASTs)**, consisting of expressions nested inside expressions:

In [160]:
ex = :( sin(3a + 2b^2) )

:(sin(3a + 2 * b ^ 2))

In [161]:
dump(ex)

Expr
  head: Symbol call
  args: Array{Any}((2,))
    1: Symbol sin
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol +
        2: Expr
          head: Symbol call
          args: Array{Any}((3,))
            1: Symbol *
            2: Int64 3
            3: Symbol a
        3: Expr
          head: Symbol call
          args: Array{Any}((3,))
            1: Symbol *
            2: Int64 2
            3: Expr
              head: Symbol call
              args: Array{Any}((3,))
                1: Symbol ^
                2: Symbol b
                3: Int64 2


We can also manipulate expressions

In [163]:
blk = quote # multi-line expression
    println("Hello")
end

quote
    #= In[163]:2 =#
    println("Hello")
end

In [164]:
eval(blk)

Hello


In [165]:
dump(blk)

Expr
  head: Symbol block
  args: Array{Any}((2,))
    1: LineNumberNode
      line: Int64 2
      file: Symbol In[163]
    2: Expr
      head: Symbol call
      args: Array{Any}((2,))
        1: Symbol println
        2: String "Hello"


In [167]:
push!(blk.args, :(println("AFTER")))

3-element Array{Any,1}:
 :(#= In[163]:2 =#) 
 :(println("Hello"))
 :(println("AFTER"))

In [168]:
blk

quote
    #= In[163]:2 =#
    println("Hello")
    println("AFTER")
end

In [169]:
eval(blk)

Hello
AFTER


In [179]:
x = 2

2

In [180]:
:(x + $x)

:(x + 2)

In [181]:
x = :y

:y

In [182]:
:(x + $x)

:(x + y)

### Use case: Programmatically define methods

In [141]:
struct MyNumber
    x::Float64
end

for op = (:sin, :cos, :tan, :log, :exp)
    eval(quote
        Base.$op(a::MyNumber) = MyNumber($op(a.x))
    end)
end

In [145]:
sin(MyNumber(pi/2))

MyNumber(1.0)

# Macros

*Macros* provide a particular use pattern of metaprogramming: replacing one expression with another, in-place, right after parsing.

Macros are useful in several cases:

to provide a specific notation different than what can normally be written in the language
e.g. https://github.com/JuliaOpt/JuMP.jl/blob/release-0.18/examples/sudoku.jl#L44
* to rearrange or delay evaluation of code
* to eliminate boilerplate (repetitive) code
* to automatically generate complex code that would be painful by hand
* to unroll loops for efficiency
* to inline code for efficiency

Macros are invoked using the @ sign, e.g.

A trivial example of defining a macro is the following, which runs whatever code it is passed two times.

In [84]:
macro twice(ex)
    quote
        $ex
        $ex
    end
end

@twice (macro with 1 method)

In [94]:
@twice println("test")

test
test


In [96]:
@macroexpand @twice println("test")

quote
    #= In[84]:3 =#
    (Main.println)("test")
    #= In[84]:4 =#
    (Main.println)("test")
end

In [97]:
@macroexpand @time 2 + 2

quote
    #= util.jl:154 =#
    local #84#stats = (Base.gc_num)()
    #= util.jl:155 =#
    local #86#elapsedtime = (Base.time_ns)()
    #= util.jl:156 =#
    local #85#val = 2 + 2
    #= util.jl:157 =#
    #86#elapsedtime = (Base.time_ns)() - #86#elapsedtime
    #= util.jl:158 =#
    local #87#diff = (Base.GC_Diff)((Base.gc_num)(), #84#stats)
    #= util.jl:159 =#
    (Base.time_print)(#86#elapsedtime, (#87#diff).allocd, (#87#diff).total_time, (Base.gc_alloc_count)(#87#diff))
    #= util.jl:161 =#
    (Base.println)()
    #= util.jl:162 =#
    #85#val
end

### [Macro Hygiene](https://docs.julialang.org/en/latest/manual/metaprogramming/#Hygiene-1)

In [124]:
macro set_z(val)
    :(z = $val)
end

@set_z (macro with 1 method)

In [131]:
@set_z(3.0)

3.0

In [132]:
z

3.0

In [133]:
@macroexpand @set_z(3.0)

:(#99#z = 3.0)

In [134]:
macro set_z(val)
    :($(esc(:z)) = $val)
end

@set_z (macro with 1 method)

In [135]:
@set_z(3.0)

3.0

In [136]:
z

3.0

In [137]:
@macroexpand @set_z(3.0)

:(z = 3.0)

In [138]:
z

3.0

# Custom string literals

In [1]:
struct KeepZerosFloat{T<:AbstractFloat}
    x::T
    n::Int64 # extra tail of n zeros
end

In [2]:
KeepZerosFloat(z::T) where {T<:AbstractFloat} = KeepZerosFloat{T}(z, 0)

KeepZerosFloat

In [3]:
KeepZerosFloat(3.0)

KeepZerosFloat{Float64}(3.0, 0)

In [57]:
Base.show(io::IO, z::KeepZerosFloat) = print(io, string(z.x) * join(fill('0', z.n)))

In [58]:
KeepZerosFloat(3.0)

3.0

In [59]:
KeepZerosFloat(3.0000)

3.0

In [77]:
macro k_str(x)
    xF64 = parse(Float64, x)
    s = string(xF64)
    zeros_str = replace(x, s => "")
    @assert zeros_str === join(fill('0', length(zeros_str)))
    
    KeepZerosFloat{Float64}(xF64, length(zeros_str))
end

@k_str (macro with 1 method)

In [80]:
k"3.0"

3.0

In [81]:
k"3.000"

3.000

In [84]:
x = k"3.1410000000"

3.1410000000

In [85]:
typeof(x)

KeepZerosFloat{Float64}

# Unrolling

There are many interesting examples of macros in Base. One that is accessible is Horner's method for evaluating a polynomial:

$$p(x) = a_n x^n + a_{n-1} x^{n-1} + \cdots + a_1 x^1 + a_0$$
may be evaluated efficiently as

$$p(x) = a_0 + x(a_1 + \cdots x(a_{n-2} + \cdots + x(a_{n-1} + x a_n) \cdots ) ) $$
with only $n$ multiplications.

The obvious way to do this is with a for loop. But if we know the polynomial at **compile time**, this loop may be unrolled using metaprogramming. This is implemented in the Math module in math.jl in Base, so the name of the macro (which is not exported) is `@Base.Math.horner`.

In [44]:
function horner_loop(x, a...)
    r = a[end]
    for i in length(a)-1:-1:1
        r = a[i] + x*r
    end
    r
end

horner_loop (generic function with 1 method)

In [45]:
horner_loop(x, 2, 3, 4, 5)

182

### Macro version

In [ ]:
# Modified from base/math.jl
macro horner(x, a...)
    ex = esc(a[end])
    t = esc(x)
    for i in length(a)-1:-1:1
        ex = :( $(esc(a[i])) + $t * $ex )
    end
    ex
end

In [13]:
x = 3
@horner(x, 2, 3, 4, 5)

182

In [80]:
@macroexpand @horner(x, 2, 3, 4, 5)

:(2 + x * (3 + x * (4 + x * 5)))

In [81]:
horner(x) = @horner(x, 2, 3, 4, 5)

horner (generic function with 1 method)

### Comparison

In [82]:
@code_warntype horner(x)

Body::Int64
1 ─ %1 = (Base.mul_int)(x, 5)::Int64
│   %2 = (Base.add_int)(4, %1)::Int64
│   %3 = (Base.mul_int)(x, %2)::Int64
│   %4 = (Base.add_int)(3, %3)::Int64
│   %5 = (Base.mul_int)(x, %4)::Int64
│   %6 = (Base.add_int)(2, %5)::Int64
└──      return %6


In [83]:
@code_warntype horner_loop(x)

Body::Int64
1 ─ %1  = (Core.tuple)(2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20)::Core.Compiler.Const((2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20), false)
│         (Int64 <: Base.Signed)
└──       goto #7 if not true
2 ┄ %4  = φ (#1 => 20, #6 => %9)::Int64
│   %5  = φ (#1 => 18, #6 => %15)::Int64
│   %6  = φ (#1 => 18, #6 => %16)::Int64
│   %7  = (Base.getfield)(%1, %5, true)::Int64
│   %8  = (Base.mul_int)(x, %4)::Int64
│   %9  = (Base.add_int)(%7, %8)::Int64
│   %10 = (%6 === 1)::Bool
└──       goto #4 if not %10
3 ─       goto #5
4 ─ %13 = (Base.add_int)(%6, -1)::Int64
└──       goto #5
5 ┄ %15 = φ (#4 => %13)::Int64
│   %16 = φ (#4 => %13)::Int64
│   %17 = φ (#3 => true, #4 => false)::Bool
│   %18 = (Base.not_int)(%17)::Bool
└──       goto #7 if not %18
6 ─       goto #2
7 ┄ %21 = φ (#5 => %9, #1 => 20)::Int64
└──       goto #8
8 ─       return %21


## Unroll.jl

In [182]:
] dev https://github.com/StephenVavasis/Unroll.jl

   Cloning git-repo `https://github.com/StephenVavasis/Unroll.jl`
Fetching: [========================================>]  100.0 %.0 % %  Updating git-repo `https://github.com/StephenVavasis/Unroll.jl`
 Resolving package versions...


┌ Info: Assigning UUID bec05c2e-cc06-5df2-90c3-340a513e71ff to Unroll
└ @ Pkg.Types C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\Pkg\src\Types.jl:841


  Updating `C:\Users\carsten\Desktop\julia-workshop-2019\Project.toml`
  [bec05c2e] + Unroll v0.0.0 [`C:\Users\carsten\.julia\dev\Unroll`]
  Updating `C:\Users\carsten\Desktop\julia-workshop-2019\Manifest.toml`
  [bec05c2e] + Unroll v0.0.0 [`C:\Users\carsten\.julia\dev\Unroll`]


In [2]:
using Unroll, BenchmarkTools

┌ Info: Recompiling stale cache file C:\Users\carsten\.julia\compiled\v1.1\Unroll\GnJ37.ji for Unroll [bec05c2e-cc06-5df2-90c3-340a513e71ff]
└ @ Base loading.jl:1184


In [34]:
function f()
    z = 1.0
    for i in 1:10
        z += i*rand()
    end
    z
end

f (generic function with 1 method)

In [35]:
@time f()

  0.012787 seconds (32.05 k allocations: 1.766 MiB)


30.116107186413537

In [36]:
@code_lowered f()

CodeInfo(
1 ─       z = 1.0
│   %2  = 1:10
│         #temp# = (Base.iterate)(%2)
│   %4  = #temp# === nothing
│   %5  = (Base.not_int)(%4)
└──       goto #4 if not %5
2 ┄ %7  = #temp#
│         i = (Core.getfield)(%7, 1)
│   %9  = (Core.getfield)(%7, 2)
│   %10 = z
│   %11 = i
│   %12 = (Main.rand)()
│   %13 = %11 * %12
│         z = %10 + %13
│         #temp# = (Base.iterate)(%2, %9)
│   %16 = #temp# === nothing
│   %17 = (Base.not_int)(%16)
└──       goto #4 if not %17
3 ─       goto #2
4 ┄       return z
)

In [37]:
@btime f()

  21.642 ns (0 allocations: 0 bytes)


38.09206697201813

In [38]:
function g()
    z = 1.0
    @unroll for i in 1:10
        z += i*rand()
    end 
    z
end

g (generic function with 1 method)

In [39]:
@time g()

  0.029493 seconds (106.86 k allocations: 5.611 MiB)


41.343279050125766

In [40]:
@btime g();

  18.218 ns (0 allocations: 0 bytes)


In [41]:
@code_lowered g()

CodeInfo(
1 ─       z = 1.0
│   %2  = z
│   %3  = (Main.rand)()
│   %4  = 1 * %3
│         z = %2 + %4
│   %6  = z
│   %7  = (Main.rand)()
│   %8  = 2 * %7
│         z = %6 + %8
│   %10 = z
│   %11 = (Main.rand)()
│   %12 = 3 * %11
│         z = %10 + %12
│   %14 = z
│   %15 = (Main.rand)()
│   %16 = 4 * %15
│         z = %14 + %16
│   %18 = z
│   %19 = (Main.rand)()
│   %20 = 5 * %19
│         z = %18 + %20
│   %22 = z
│   %23 = (Main.rand)()
│   %24 = 6 * %23
│         z = %22 + %24
│   %26 = z
│   %27 = (Main.rand)()
│   %28 = 7 * %27
│         z = %26 + %28
│   %30 = z
│   %31 = (Main.rand)()
│   %32 = 8 * %31
│         z = %30 + %32
│   %34 = z
│   %35 = (Main.rand)()
│   %36 = 9 * %35
│         z = %34 + %36
│   %38 = z
│   %39 = (Main.rand)()
│   %40 = 10 * %39
│         z = %38 + %40
└──       return z
)

Based on https://github.com/mitmath/18S096/blob/master/lectures/lecture7/Metaprogramming.ipynb